In [1]:
import numpy as np
import os
import math
import random
import torch
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from Model import RNN, CNN
from tqdm import tqdm_notebook

### Prepare the data

In [2]:
EVAL_DATA_DIR = os.path.abspath(os.path.abspath('') + '/Aniyama_groundtruth')

In [3]:
data_types = [p for p in os.listdir(EVAL_DATA_DIR) if not p.startswith('.')]
dataset = []
for t in data_types:
    type_dir = f'{EVAL_DATA_DIR}/{t}'
    for file in os.listdir(type_dir):
        filename = f'{type_dir}/{file}'
        examples = []
        with open(filename, 'r') as f:
            for i, line in enumerate(f.readlines()):
                if i == 0:
                    continue
                tokens = line.rstrip().split(',')
                power, anomaly = float(tokens[1]), int(tokens[2])
                example = [power, anomaly]
                examples.append(example)
    examples = np.array(examples)
    dataset.append(examples)

In [4]:
data = []
labels = []
for chunk in dataset:
    size = chunk.shape[0]
    new_size = (math.ceil(size / 10) * 10)
    pad_size = new_size - size
    padding = np.zeros((pad_size, 2))
    new_chunk = np.vstack((chunk, padding)).reshape(new_size // 10, 10, 2)
    for i in range(new_chunk.shape[0]):
        data.append(new_chunk[i,:,0])
        label = new_chunk[i,:,1].sum() > 0
        labels.append(float(label))
data = np.array(data)
labels = np.array(labels)

In [5]:
data.shape, labels.shape

((3429, 10), (3429,))

In [6]:
labels.sum() / len(labels)

0.49518810148731407

In [7]:
X_train, X_test, y_train, y_text = train_test_split(data, labels, test_size = 0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train, X_valid, X_test = torch.FloatTensor(X_train), torch.FloatTensor(X_train), torch.FloatTensor(X_train)
y_train, y_valid, y_test = torch.FloatTensor(y_train), torch.FloatTensor(y_train), torch.FloatTensor(y_train)

### Setup Data Loader

In [9]:
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, data, labels):
        'Initialization'
        self.labels = labels
        self.data = data

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        return self.data[index], self.labels[index]

In [10]:
config = {
    'batch_size': 32,
    'shuffle': True,
    'num_workers': 6,
    'drop_last': True,
    'num_epochs': 400,
    'encode_dim': 1,
    'hidden_dim': 128,
    'output_dim': 1,
    'num_layers': 3,
    'dropout': 0.2,
    'device': device
}

In [11]:
train_set = Dataset(X_train, y_train)
train_loader = data.DataLoader(
    train_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

validation_set = Dataset(X_valid, y_valid)
validation_loader = data.DataLoader(
    validation_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

test_set = Dataset(X_test, y_test)
test_loader = data.DataLoader(
    test_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

In [12]:
models = {}
models['baseline_rnn'] = RNN(
    config['encode_dim'],
    config['hidden_dim'],
    config['output_dim'],
    config['num_layers'],
    config['dropout']
)

In [13]:
model = models['baseline_rnn']

### TensorBoard Setup

In [14]:
writer = SummaryWriter('runs/baseline_rnn/')

### Train/Validate the Model

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 331,393 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

model = model.to(config['device'])
criterion = criterion.to(config['device'])

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [18]:
def train(model, loader, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_data, batch_labels in loader:
        
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
                
        optimizer.zero_grad()
        
        predictions = model(batch_data).squeeze(1)
        
        loss = criterion(predictions, batch_labels)
        
        acc = binary_accuracy(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [19]:
def evaluate(model, loader, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_data, batch_labels in loader:
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            predictions = model(batch_data).squeeze(1)
            
            loss = criterion(predictions, batch_labels)
            
            acc = binary_accuracy(predictions, batch_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [20]:
def test(model, loader, device):
    epoch_acc = 0
    with torch.no_grad():
    
        for batch_data, batch_labels in loader:
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            predictions = model(batch_data).squeeze(1)
            acc = binary_accuracy(predictions, batch_labels)

            epoch_acc += acc.item()
        
    return epoch_acc / len(loader)

In [21]:
best_valid_loss = float('inf')

for epoch in tqdm_notebook(range(config['num_epochs'])):
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, config['device'])
    valid_loss, valid_acc = evaluate(model, validation_loader, criterion, config['device'])
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn-base-model.pt')
    
    # log the running loss
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', valid_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/validation', valid_acc, epoch)

In [22]:
test(model, test_loader, config['device'])

0.92578125

### CNN

In [23]:
writer = SummaryWriter('runs/baseline_cnn/')
models['baseline_cnn'] = CNN()
model = models['baseline_cnn']

In [24]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,074 trainable parameters


In [25]:
optimizer = optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

model = model.to(config['device'])
criterion = criterion.to(config['device'])

In [26]:
best_valid_loss = float('inf')

for epoch in tqdm_notebook(range(config['num_epochs'])):
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, config['device'])
    valid_loss, valid_acc = evaluate(model, validation_loader, criterion, config['device'])
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'cnn-base-model.pt')
    
    # log the running loss
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', valid_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/validation', valid_acc, epoch)

In [27]:
test(model, test_loader, config['device'])

0.8779296875

### Clustering

In [115]:
RNN = models['baseline_rnn']

In [116]:
examples = X_test.numpy()

In [117]:
examples.shape

(2057, 10)

In [118]:
embeddings = RNN.encode(X_test.to(config['device'])).data.cpu().numpy()

In [119]:
embeddings.shape

(2057, 128)

In [120]:
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn import mixture
import matplotlib.pyplot as plt
from collections import defaultdict

In [121]:
kmeans = KMeans(n_clusters=20, random_state=72).fit(embeddings)

In [122]:
kmeans.labels_, kmeans.cluster_centers_

(array([ 2, 13, 18, ..., 13, 16, 14], dtype=int32),
 array([[ 0.29701334, -0.07777032, -0.1431145 , ..., -0.15044579,
         -0.06372637,  0.20957312],
        [ 0.60893077, -0.16159454, -0.08901623, ...,  0.11905408,
         -0.00322957,  0.04911911],
        [-0.10350417, -0.02464201, -0.02748109, ...,  0.2399875 ,
         -0.05707791,  0.06218136],
        ...,
        [ 0.16245586, -0.05258185, -0.00112937, ...,  0.20682925,
         -0.16620062, -0.04883839],
        [-0.25816345, -0.0235463 ,  0.01080968, ...,  0.26803938,
         -0.02965311,  0.06734238],
        [-0.02668024,  0.8527755 ,  0.01289859, ...,  0.2505462 ,
         -0.01151062,  0.45533922]], dtype=float32))

In [123]:
kmeans.cluster_centers_.shape

(20, 128)

In [124]:
def kmeans_distance(index):
    return distance.euclidean(embeddings[index], kmeans.cluster_centers_[kmeans.labels_[index]])

In [125]:
center_distances = defaultdict(list)
for i in range(len(embeddings)):
    center_distances[kmeans.labels_[i]].append((kmeans_distance(i), i))
center_distances = {k: sorted(v, key=lambda x: x[0], reverse=False) for k, v in center_distances.items()}

In [126]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [140]:
for label, encodings in center_distances.items():
    fig = go.Figure()
    for i in range(3):
        distance, index = encodings[i]
        y = X_test[index]
        fig.add_trace(
            go.Scatter(
                x=np.arange(len(y)), 
                y=y,
                mode='lines',
                name=f"Rank {i+1}|Distance: {distance:.2f}"))
        
    fig.update_layout(
        title=f"Label {label}",
        xaxis_title="window index",
        yaxis_title="power value",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    config = {"toImageButtonOptions": {"width": None, "height": None}}
    fig.show(config=config)
    fig.write_image(f"images/cluster_label_{label}.png")